In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql

#### 모든 제품을 크롤링해서 DB에 저장
- 저장할 DB부터 테이블 생성해서 형식에 맞게 data 변환 후 DB로 insert
1. DB 연결코드
2. 필요한 DB 및 table 생성
3. 크롤링
    - 데이터를 table 형식에 맞게 정제
4. insert (데이터 생성)
5. commit() 해서 db에 반영
6. db 닫기

- db 테이블을 읽어와서 df에 저장



In [3]:
def conn(d_name) :
    import pymysql
    host_name = 'localhost'
    host_port = 3306
    username = 'root'
    password = '0000'
    database_name = d_name

    # db  연결
    db = pymysql.connect(
        host=host_name,     # MySQL Server Address
        port=host_port,          # MySQL Server Port
        user=username,      # MySQL username
        passwd=password,    # password for MySQL username
#         db=database_name,   # Database name
        charset='utf8'
    )
    return db


In [5]:
db = conn('none')
cursor = db.cursor()


In [6]:
sql = 'create database beauty_shop'
cursor.execute(sql)

1

In [7]:
sql = "show databases"
cursor.execute(sql)
result = cursor.fetchall()
result



(('beauty_shop',),
 ('customer_db',),
 ('dave',),
 ('ecommerce',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sqldb',),
 ('student_mgmt',),
 ('test',))

In [8]:
sql = "use beauty_shop"
cursor.execute(sql)


0

In [10]:
sql = "select database()"
cursor.execute(sql)
result = cursor.fetchone()
result


('beauty_shop',)

In [14]:
# table 생성
sql = '''
    CREATE TABLE product (
        PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
        TITLE VARCHAR(200) NOT NULL,
        ORI_PRICE FLOAT,
        DISCOUNT_PRICE FLOAT,
        link VARCHAR(200),
        PRIMARY KEY(PRODUCT_CODE)
    );
'''

cursor.execute(sql)
db.commit()

In [15]:
sql = "show tables"
cursor.execute(sql)
result = cursor.fetchall()
result


(('product',),)

In [17]:
sql = 'desc product'
cursor.execute(sql)
result = cursor.fetchall()
result


(('PRODUCT_CODE', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('TITLE', 'varchar(200)', 'NO', '', None, ''),
 ('ORI_PRICE', 'float', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'float', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

2. 크롤링 코드 - insert 구문을 추가해서 변경



In [18]:
# 크롤링 문서 요청해서 응답객체 반환(첫번째 크롤링 문서 요청 후 응답)
url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()
# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")

In [19]:
# box안에 들어 있는 1개의 상품에서 정보를 추출해서 dict형태로  반환하는 함수
# 데이터 전처리 :  제품명에 ' 제거 / 가격  USD 제거/ 세일 가격 없는 경우 처리

def get_product_info(box) :
    p_tag = box.find("p",{"class":"name"})
    # 품목 추출
    span = p_tag.find("span")
    
    # 세부페이지링크 추출
    a = p_tag.find("a")
    sub_link = 'https://jolse.com' + a["href"]
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = price_ul.findAll("span")
    
    # 데이터 전처리
    title = span.text.replace("'","''") # ' 처리
    ord_price = price_span[1].text.split(' ')[1] # USD 제거
    dis_price = price_span[-1].text.split(' ')[1]
    
    # 세일 가격이 없는 경우
    if dis_price == '':
        dis_price = '0.0'
        
    
    # 최종 data 추출 후 반환    
    return{"prd_name":title,"price":ord_price,"sale_price":dis_price,"sub_link":sub_link}


In [ ]:
def save_data(prd_info) :
    print(prd_info)

In [23]:
# 전달된 url 페이지에 접근해서 해당페지의 전체 상품 데이터를 추출 한 후 각 상품마다 get_product_info()함수를 호출해서
# 각 상품에대한 추출 정보를 받아옴 - 들어온 각 상품 정보를 리스트에 저장 한 수 해당 반환
def get_page_products(url) :
    url=url
    html = urlopen(url)
    htmls = html.read()
    # print(htmls)
    bs_obj = BeautifulSoup(htmls,"html.parser")

    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"prdList grid4"})
    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    prd_boxes = ul.findAll("div", {"class":"box"}) #1개 페이지의 전체 상품
    
    # 반환되는 품목 데이터를 db에 insert : 함수 호출해서 진행
    for box in prd_boxes :
        prd = get_product_info(box)
#         print(prd)
        save_data(prd) # 사용자 정의 함수 생성해야함 
        

In [24]:
#last=
int(bs_obj.find("p",{"class":"last"}).find("a")['href'].split("=")[1])

12

## 메인코드


In [25]:
from tqdm import tqdm_notebook # 상태바 표시

#여러 페이지의 화장품 정보를 추출해서 df 에 저장 후 csv에 저장하는 코드
url = "http://jolse.com/category/toners-mists/1019/?page=" #페이지 번호를 제외한 공통 url 문자열

last= int(bs_obj.find("p",{"class":"last"}).find("a")['href'].split("=")[1])

for i in range(1,2) : 
    # url 완성 :page번호를 추가 i 변수 값을 활용
    urlfin =url + str(i)
    get_page_products(urlfin)
  

{'prd_name': 'Isntree Hyaluronic Acid Toner 200ml (Renewal)', 'price': 'USD 19.00', 'sale_price': 'USD 13.99', 'sub_link': 'https://jolse.com/product/detail.html?product_no=22560&cate_no=1019&display_group=1'}
{'prd_name': 'SON&PARK BEAUTY WATER 340ml', 'price': 'USD 29.40', 'sale_price': 'USD 24.99', 'sub_link': 'https://jolse.com/product/detail.html?product_no=10953&cate_no=1019&display_group=1'}
{'prd_name': 'SOME BY MI AHA BHA PHA 30 Days Miracle Toner 150ml', 'price': 'USD 24.00', 'sale_price': 'USD 20.40', 'sub_link': 'https://jolse.com/product/detail.html?product_no=16241&cate_no=1019&display_group=1'}
{'prd_name': 'SOME BY MI Galactomyces Pure Vitamin C Glow Toner 200ml', 'price': 'USD 34.00', 'sale_price': 'USD 28.90', 'sub_link': 'https://jolse.com/product/detail.html?product_no=19667&cate_no=1019&display_group=1'}
{'prd_name': 'PURITO Centella Green Level Calming Toner 200ml', 'price': 'USD 25.20', 'sale_price': 'USD 16.99', 'sub_link': 'https://jolse.com/product/detail.html

In [26]:
 
df_fin

NameError: name 'df_fin' is not defined

In [18]:
df_fin.to_csv('./crawl_data/prod_All.csv')